# Unlabeled Data Analysis

create twitter hashtag reader and reddit thread reader

pick sample hashtags and threads

In [1]:
import nest_asyncio
nest_asyncio.apply()
import predictions

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
pos_percentage, neg_percentage, pos_samples, neg_samples = predictions.get_web_output('#pokemonpresents')

In [12]:
pos_percentage, neg_percentage

(96, 4)

In [13]:
pos_samples

['The #SPAWNCAST is LIVE discussing: #PokemonPresents, #PlayStation State of Play, #PokemonLegendsArceus | ft. @RGT_85 @ModernVintageG @NateTheHate2 @SpawnWaveMedia @PlayerEssence @_MissClick_ @KimerexProjekt @JordanFringe94 @AustinJohnPlays @DirectFeedGames  https://t.co/JEOIJYY9CP',
 'My first set of Luxery Gamer Press-Ons 😍 Jiggly Puff inspired by the #PokemonPresents today 💖💖 🎶  https://t.co/0CAu1Fp78N',
 "#PokemonPresents LES REMAKES LET'S GOOOOOO  https://t.co/RMOswSTOGh",
 "As #PokemonPresents is trending,  I'm throwing out the ACEO Card trio of my favorite Pokemon!  https://t.co/FBlVslrIkP",
 'I knew I would be sick by this day, so I prepared this drawing months ago Happy #PokemonDay! . . . . Support me:  https://t.co/7hw3GaOyEV #furry #PokemonPresents #cute #ArtistOnTwitter #art  https://t.co/393yY5b1F8']

In [14]:
neg_samples

["having the games in a chibi style is not that much of a problem but for my liking they should've gone for a 3-heads-tall style (like Let's GO) instead of 2-heads-tall  #PokemonPresents #Pokemon25 #PokemonDiamondPearl  https://t.co/536YnxpRZh",
 "I'm pretty confident I can make Arceus (ar-kay-us now apparently??) look like an absolute terror  #PokemonPresents #PokemonDirect #PokemonDay  https://t.co/ek6AInNzQY",
 'The Pokémon Company dropping two Sinnoh games today  #PokemonPresents    https://t.co/T7wwuFxWNn',
 "Missed #PokemonPresents #Pokemon25 presentation earlier? We've got you covered from trailers, new game announcements and more.  https://t.co/dtkJ7dOE1W",
 "I wonder what's going to be announced in the #PokemonPresents video they made for #PokemonDay I like #Pokemon and #PokemonGo and #SonicTheHedgehog and stuffs  https://t.co/fSfGZADYek"]

sample reddit thread ideas:
- https://www.reddit.com/r/nintendo/comments/lm6obv/project_triangle_strategy_announcement_trailer/
- https://www.reddit.com/r/nintendo/comments/lru33p/the_animal_crossing_new_horizons_free_update_is/
- https://www.reddit.com/r/nintendo/comments/lqlrqg/tony_hawks_pro_skater_1_2_coming_to_nintendo/
- https://www.reddit.com/r/Games/comments/ls6qlh/bravery_default_ii_review_thread/

sample hashtags:
- #stateofplay
- #pokemonpresents

run data through processing and model

output class percentages and numbers

output unlabeled topic analysis

CREATE FRONTEND!!!